# Azure AI Search са интеграцијом NVIDIA NIM и LlamaIndex

У овом нотебуку, демонстрираћемо како да искористимо NVIDIA AI моделе и LlamaIndex за креирање моћног Retrieval-Augmented Generation (RAG) система. Користићемо NVIDIA LLM-ове и ембедингс, интегрисати их са Azure AI Search као векторским складиштем, и извршити RAG ради побољшања квалитета и ефикасности претраге.

## Предности
- **Скалабилност**: Искористите велике језичке моделе компаније NVIDIA и Azure AI Search за скалабилно и ефикасно преузимање.
- **Ефикасност трошкова**: Оптимизујте претрагу и преузимање уз ефикасно складиштење вектора и хибридне технике претраге.
- **Високе перформансе**: Комбинујте моћне LLM-ове са векторизованом претрагом за брже и прецизније одговоре.
- **Квалитет**: Одржавајте висок квалитет претраге уз ослањање на релевантне документе који су преузети.

## Предуслови
- 🐍 Python 3.9 или новији
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API кључ за приступ NVIDIA LLM-овима и ембедингсима преко NVIDIA NIM микросервиса

## Покривене функције
- ✅ Интеграција NVIDIA LLM-ова (користићемо [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA ембедингс (користићемо [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Напредни режими преузимања Azure AI Search-а
- ✅ Индексирање докумената уз LlamaIndex
- ✅ RAG уз Azure AI Search и LlamaIndex са NVIDIA LLM-овима

Хајде да почнемо!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Инсталација и Захтеви
Креирајте Python окружење користећи Python верзију >3.10.

## Почетак!


Да бисте започели, потребан вам је `NVIDIA_API_KEY` за коришћење модела NVIDIA AI Foundation:  
1) Направите бесплатан налог на [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Кликните на модел који желите да користите.  
3) У оквиру Input, изаберите Python картицу, затим кликните на **Get API Key**, а потом на **Generate Key**.  
4) Копирајте и сачувајте генерисани кључ као NVIDIA_API_KEY. Од тог тренутка, требало би да имате приступ ендпоинтима.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Пример RAG користећи LLM и Уграђивање
### 1) Иницијализујте LLM
`llama-index-llms-nvidia`, познат и као NVIDIA-ов LLM конектор, омогућава вам да се повежете са компатибилним моделима и генеришете садржај користећи моделе доступне у NVIDIA API каталогу. Погледајте овде листу модела за завршетак разговора: https://build.nvidia.com/search?term=Text-to-Text

Овде ћемо користити **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Иницијализујте Уграђивање
`llama-index-embeddings-nvidia`, такође познат као NVIDIA-ов конектор за уграђивања, омогућава вам да се повежете са компатибилним моделима и генеришете из њих, доступним у NVIDIA API каталогу. Изабрали смо `nvidia/nv-embedqa-e5-v5` као модел за уграђивање. Погледајте овде листу модела за текстуално уграђивање: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Креирајте Azure AI Search Vector Store


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Направите механизам за упите како бисте постављали питања о вашим подацима

Овде је пример упита који користи чисту претрагу вектора у Azure AI Search и повезује одговор са нашим LLM-ом (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Ево упита који користи хибридну претрагу у Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Анализа претраге вектора
Одговор LLM-а тачно обухвата кључне економске последице поменуте у изворном тексту у вези са руском берзом. Конкретно, наводи да је руска берза доживела значајан пад, изгубивши 40% своје вредности, и да је трговање обустављено због актуелне ситуације. Овај одговор се добро поклапа са информацијама из извора, указујући на то да је LLM исправно идентификовао и сумирао релевантне детаље о утицају на берзу као резултат руских акција и уведених санкција.

#### Коментар на изворне чворове
Изворни чворови пружају детаљан приказ економских последица које је Русија претрпела због међународних санкција. Текст истиче да је руска берза изгубила 40% своје вредности и да је трговање обустављено. Поред тога, помињу се и друге економске последице, као што су девалвација рубље и шира изолација руске економије. Одговор LLM-а је ефикасно издвојио кључне тачке из ових чворова, фокусирајући се на утицај на берзу, као што је захтевано у упиту.


Хајде да погледамо упит где Хибридна претрага не даје добро утемељен одговор:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Хибридна претрага: Анализа одговора LLM-а
Одговор LLM-а у примеру хибридне претраге указује на то да дати контекст не наводи тачан датум руске инвазије на Украјину. Овај одговор сугерише да LLM користи информације доступне у изворним документима, али истовремено признаје одсуство прецизних детаља у тексту.

Одговор је тачан у идентификовању да контекст помиње догађаје повезане са руском агресијом, али не прецизира тачан датум инвазије. Ово показује способност LLM-а да разуме пружене информације, истовремено препознајући празнине у садржају. LLM ефикасно подстиче корисника да потражи спољне изворе или историјске записе за тачан датум, показујући одређени ниво опреза када су информације непотпуне.

### Анализа изворних чворова
Изворни чворови у примеру хибридне претраге садрже одломке из говора који разматра одговор САД на руске акције у Украјини. Ови чворови наглашавају шири геополитички утицај и кораке које су САД и њени савезници предузели као одговор на инвазију, али не помињу тачан датум инвазије. Ово је у складу са одговором LLM-а, који исправно идентификује да контексту недостају прецизни подаци о датуму.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Хибрид са поновним рангирањем: Анализа одговора LLM-а
У примеру Хибрид са поновним рангирањем, одговор LLM-а пружа додатни контекст напомињући да се догађај догодио шест дана пре него што је одржан говор. Ово указује на то да LLM може да закључи датум инвазије на основу времена одржавања говора, иако му је и даље потребно знати тачан датум говора ради прецизности.

Овај одговор показује побољшану способност коришћења контекстуалних назнака за пружање информативнијег одговора. Истиче предност поновног рангирања, где LLM може да приступи релевантнијим информацијама и да их приоритизује како би дао приближнији одговор на тражени детаљ (тј. датум инвазије).

### Анализа изворних чворова
Изворни чворови у овом примеру укључују референце на време руске инвазије, конкретно помињући да се догодила шест дана пре говора. Иако тачан датум и даље није експлицитно наведен, чворови пружају временски контекст који омогућава LLM-у да да суптилнији одговор. Укључивање овог детаља показује како поновно рангирање може побољшати способност LLM-а да извуче и закључи информације из датог контекста, што резултира прецизнијим и информативнијим одговором.


**Напомена:**
У овом нотебуку користили смо NVIDIA NIM микросервисе из NVIDIA API каталога. 
Горепоменути API-ји, `NVIDIA (llms)`, `NVIDIAEmbedding`, и [Azure AI Search Semantic Hybrid Retrieval (уграђено поновно рангирање)](https://learn.microsoft.com/azure/search/semantic-search-overview). Напомена, да горепоменути API-ји такође могу подржавати самостално хостоване микросервисе.

**Пример:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.
